In [8]:
# GENERAL IMPORTS

import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='lnumerow', api_key='mDhzegrhH0qf8rlFbzOo')
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

import datetime
import math

In [9]:
datafile = r'''Data\IVE_bidask1min.txt'''

colnames = ['Date', 'Time', 'BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskLow', 'AskHigh', 'AskClose']
fullpricedata = pd.read_csv(datafile, names=colnames)

fullpricedata['DateTime'] = (fullpricedata['Date']+fullpricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
del fullpricedata['Date']
del fullpricedata['Time']

fullpricedata = fullpricedata[[(dt >= datetime.datetime(dt.year, dt.month, dt. day, 9, 30, 0)) and (dt <= datetime.datetime(dt.year, dt.month, dt. day, 16, 0, 0)) for dt in fullpricedata['DateTime']]].copy()

In [14]:
pricedata = fullpricedata[fullpricedata['DateTime'] > datetime.datetime(2019,1,1,0,0,0)].copy()
pricedata = pricedata.reset_index()
del pricedata['index']

In [15]:
def changeTimeframe(pricedata, timeframe_mins):
    newpricedata = pd.DataFrame()
    
    index = 0
    
    while(index < len(pricedata)):
        newcandle = pricedata.iloc[index].copy()
        starttime = newcandle.DateTime
        
        while(True):
            index = index + 1
            if index == len(pricedata):
                break
            
            candle = pricedata.iloc[index].copy()
            
            if candle.DateTime - starttime >= datetime.timedelta(minutes=timeframe_mins):
                break
                
            newcandle.AskClose = candle.AskClose
            newcandle.AskHigh = max(newcandle.AskHigh, candle.AskHigh)
            newcandle.AskLow = min(newcandle.AskLow, candle.AskLow)
            newcandle.BidClose = candle.BidClose
            newcandle.BidHigh = max(newcandle.BidHigh, candle.BidHigh)
            newcandle.BidLow = min(newcandle.BidLow, candle.BidLow)
        
        newpricedata = newpricedata.append(newcandle)
        
    return newpricedata

In [16]:
class CandlestickChart:
    
    def updateChart(self, placeholder):
        starttime = datetime.datetime.combine(self.startdatewidget.value, datetime.time())
        endtime = datetime.datetime.combine(self.enddatewidget.value, datetime.time()) + datetime.timedelta(days=1)
        timeframe = self.timeframewidget.value
        
        fullpricedata = self.fullpricedata
        chartdata = fullpricedata[fullpricedata['DateTime'] >= starttime]
        chartdata = fullpricedata[fullpricedata['DateTime'] < endtime]
        chartdata = changeTimeframe(chartdata, timeframe).copy()
        
        data = self.chart.data[0]
        data.x = [ii for ii in range(len(chartdata['BidLow']))]
        data.low = chartdata['BidLow']
        data.open = chartdata['BidOpen']
        data.close = chartdata['BidClose']
        data.high = chartdata['BidHigh']
        
        layout = self.chart.layout
        layout.xaxis.tickvals = [ii for ii in data.x if 
                                 chartdata.iloc[ii]['DateTime'].time() == datetime.time(9, 30)]
        xaxisdatetimes = [chartdata.iloc[ii]['DateTime'] for ii in layout.xaxis.tickvals]
        layout.xaxis.ticktext = [dt.strftime(' %y/%m/%d ') if dt.time() == datetime.time(9, 30)
                                 else dt.strftime(' %H:%M ') for dt in xaxisdatetimes]
        
        layout.xaxis.hoverformat = ''
    
    def __init__(self, fullpricedata):
        self.fullpricedata = fullpricedata
        
        data = [ dict(
                type = 'candlestick')
               ]
        layout = dict(
                title = 'Candlestick Chart',
                xaxis = dict(
                    title = dict(
                        text = 'Time'
                    )
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Price'
                    )
                ))
        self.chart = go.FigureWidget( data=data, layout=layout)
        
        # DEFINE WIDGETS AND CALLBACK FUNCTIONS
        self.startdatewidget = ipywidgets.DatePicker(
            description='Start Date',
            value=datetime.date(2019,1,2),
            disabled=False
        )
        self.enddatewidget = ipywidgets.DatePicker(
            description='End Date',
            value=datetime.date(2019,1,2),
            disabled=False
        )
        self.timeframewidget = ipywidgets.IntText(
            description='Timeframe (minutes)',
            value=1,
            disabled=False
        )
        self.widgets = [self.startdatewidget, self.enddatewidget, self.timeframewidget]
        for widget in self.widgets:
            widget.observe(self.updateChart, names = 'value')
        
        self.widgetsdisplay = ipywidgets.VBox([ipywidgets.HBox([self.startdatewidget, self.enddatewidget]),
                                        self.timeframewidget])
        
        # DEFINE DISPLAY CONFIGURATION
        self.display = ipywidgets.VBox([self.chart, self.widgetsdisplay])
        
        self.updateChart(0)

In [17]:
candle = CandlestickChart(pricedata)
candle.display

    'data': [{'close': array([ 99.66,  99.78,  99.75, ..., 101.38, 101.54, 101.4…

In [28]:
cash = {'High':10000,'Low':10000,'Open':10000,'Close':10000}
shares = {'High':0,'Low':0,'Open':0,'Close':0}
shares_value = {'High':0,'Low':0,'Open':0,'Close':0}
net_value = dict(cash)

cash_list = list()
shares_list = list()
shares_value_list = list()
net_value_list = list()

start_idx = pricedata.first_valid_index();
commission = 0
threshold = 0.01
scale = 1

for index, row in pricedata.iterrows():
    buy = 0
    sell = 0
    
    if index > start_idx:
        print(index)
        last_row = pricedata.loc[index-1]
    else:
        print(index)
        last_row = pricedata.loc[index]
        
    delta = row['BidOpen'] - last_row['BidClose']
    
    if delta > threshold:
        buy = 1
        quantity = scale*round(100*delta)
    elif delta < -threshold:
        sell = 1
        quantity = scale*round(-100*delta)
    
    for k in cash.keys():
        bid_price = row['Bid' + str(k)]
        ask_price = row['Ask' + str(k)]
        if buy and cash[k] > commission:
            print('Buy!')
            cost = quantity*ask_price
            cost = min(cost, cash[k])
            quantity = cost/ask_price
            cash[k] -= cost
            shares[k] += quantity
        elif sell and shares[k] > 0:
            print('Sell!')
            quantity = min(quantity, shares[k])
            cash[k] += quantity*bid_price
            shares[k] -= quantity
        cash[k] -= commission
        
        # Careful with this rounding
        cash[k] = round(cash[k]*100)/100
        shares_value[k] = round(bid_price*shares[k])
        net_value[k] = round(cash[k] + shares_value[k])
        
    cash_list.append(dict(cash))
    shares_list.append(dict(shares))
    shares_value_list.append(dict(shares_value))
    net_value_list.append(dict(net_value))        
    print('Cash: ' + str(cash['Open']) + ', Shares: ' + str(shares['Open']) + ', Net Value: ' + str(net_value['Open']))

0
Cash: 10000.0, Shares: 0, Net Value: 10000
1
Buy!
Buy!
Buy!
Buy!
Cash: 9201.12, Shares: 8.0, Net Value: 9999.0
2
Sell!
Sell!
Sell!
Sell!
Cash: 9500.37, Shares: 5.0, Net Value: 9999.0
3
Sell!
Sell!
Sell!
Sell!
Cash: 9998.82, Shares: 0.0, Net Value: 9999.0
4
Buy!
Buy!
Buy!
Buy!
Cash: 9100.53, Shares: 9.0, Net Value: 9998.0
5
Sell!
Sell!
Sell!
Sell!
Cash: 9498.81, Shares: 5.0, Net Value: 9997.0
6
Buy!
Buy!
Buy!
Buy!
Cash: 9100.09, Shares: 9.0, Net Value: 9996.0
7
Buy!
Buy!
Buy!
Buy!
Cash: 8002.62, Shares: 20.0, Net Value: 9998.0
8
Buy!
Buy!
Buy!
Buy!
Cash: 7403.58, Shares: 26.0, Net Value: 9999.0
9
Buy!
Buy!
Buy!
Buy!
Cash: 6904.63, Shares: 31.0, Net Value: 9997.0
10
Buy!
Buy!
Buy!
Buy!
Cash: 6705.03, Shares: 33.0, Net Value: 9996.0
11
Buy!
Buy!
Buy!
Buy!
Cash: 5807.01, Shares: 42.0, Net Value: 9996.0
12
Buy!
Buy!
Buy!
Buy!
Cash: 5407.53, Shares: 46.0, Net Value: 10001.0
13
Buy!
Buy!
Buy!
Buy!
Cash: 5207.67, Shares: 48.0, Net Value: 10002.0
14
Cash: 5207.67, Shares: 48.0, Net Value: 100

Sell!
Sell!
Sell!
Sell!
Cash: 915.97, Shares: 90.91413352189562, Net Value: 10168.0
241
Cash: 915.97, Shares: 90.91413352189562, Net Value: 10160.0
242
Sell!
Sell!
Sell!
Sell!
Cash: 1322.45, Shares: 86.91413352189562, Net Value: 10154.0
243
Buy!
Buy!
Buy!
Buy!
Cash: 915.81, Shares: 90.91413352189562, Net Value: 10157.0
244
Cash: 915.81, Shares: 90.91413352189562, Net Value: 10155.0
245
Sell!
Sell!
Sell!
Sell!
Cash: 1119.05, Shares: 88.91413352189562, Net Value: 10154.0
246
Buy!
Buy!
Buy!
Buy!
Cash: 814.1, Shares: 91.91413352189562, Net Value: 10155.0
247
Buy!
Buy!
Buy!
Buy!
Cash: 407.34, Shares: 95.91413352189562, Net Value: 10159.0
248
Sell!
Sell!
Sell!
Sell!
Cash: 813.86, Shares: 91.91413352189562, Net Value: 10155.0
249
Cash: 813.86, Shares: 91.91413352189562, Net Value: 10157.0
250
Buy!
Buy!
Buy!
Buy!
Cash: 610.54, Shares: 93.91413352189562, Net Value: 10156.0
251
Cash: 610.54, Shares: 93.91413352189562, Net Value: 10158.0
252
Cash: 610.54, Shares: 93.91413352189562, Net Value: 101

Cash: 2126.58, Shares: 78.88261884787553, Net Value: 10094.0
361
Cash: 2126.58, Shares: 78.88261884787553, Net Value: 10099.0
362
Cash: 2126.58, Shares: 78.88261884787553, Net Value: 10098.0
363
Cash: 2126.58, Shares: 78.88261884787553, Net Value: 10107.0
364
Cash: 2126.58, Shares: 78.88261884787553, Net Value: 10102.0
365
Sell!
Sell!
Sell!
Sell!
Cash: 2328.5, Shares: 76.88261884787553, Net Value: 10090.0
366
Buy!
Buy!
Buy!
Buy!
Cash: 2227.51, Shares: 77.88261884787553, Net Value: 10093.0
367
Cash: 2227.51, Shares: 77.88261884787553, Net Value: 10095.0
368
Buy!
Buy!
Buy!
Buy!
Cash: 2025.33, Shares: 79.88261884787553, Net Value: 10099.0
369
Cash: 2025.33, Shares: 79.88261884787553, Net Value: 10100.0
370
Sell!
Sell!
Sell!
Sell!
Cash: 2126.29, Shares: 78.88261884787553, Net Value: 10090.0
371
Cash: 2126.29, Shares: 78.88261884787553, Net Value: 10104.0
372
Sell!
Sell!
Sell!
Sell!
Cash: 2429.68, Shares: 75.88261884787553, Net Value: 10104.0
373
Cash: 2429.68, Shares: 75.88261884787553, Ne

541
Cash: 4109.75, Shares: 59.0, Net Value: 10014.0
542
Buy!
Buy!
Buy!
Buy!
Cash: 3709.23, Shares: 63.0, Net Value: 10016.0
543
Cash: 3709.23, Shares: 63.0, Net Value: 10013.0
544
Cash: 3709.23, Shares: 63.0, Net Value: 10015.0
545
Buy!
Buy!
Buy!
Buy!
Cash: 3207.63, Shares: 68.0, Net Value: 10027.0
546
Buy!
Buy!
Buy!
Buy!
Cash: 2906.64, Shares: 71.0, Net Value: 10030.0
547
Sell!
Sell!
Sell!
Sell!
Cash: 3207.51, Shares: 68.0, Net Value: 10028.0
548
Sell!
Sell!
Sell!
Sell!
Cash: 3307.79, Shares: 67.0, Net Value: 10027.0
549
Cash: 3307.79, Shares: 67.0, Net Value: 10015.0
550
Sell!
Sell!
Sell!
Sell!
Cash: 3507.95, Shares: 65.0, Net Value: 10013.0
551
Buy!
Buy!
Buy!
Buy!
Cash: 3307.75, Shares: 67.0, Net Value: 10012.0
552
Buy!
Buy!
Buy!
Buy!
Cash: 3107.41, Shares: 69.0, Net Value: 10017.0
553
Buy!
Buy!
Buy!
Buy!
Cash: 2907.07, Shares: 71.0, Net Value: 10018.0
554
Buy!
Buy!
Buy!
Buy!
Cash: 2406.12, Shares: 76.0, Net Value: 10018.0
555
Buy!
Buy!
Buy!
Buy!
Cash: 1804.56, Shares: 82.0, Net Val

Sell!
Sell!
Sell!
Sell!
Cash: 1693.17, Shares: 82.98331289562907, Net Value: 9954.0
651
Sell!
Sell!
Sell!
Sell!
Cash: 2489.73, Shares: 74.98331289562907, Net Value: 9956.0
652
Sell!
Sell!
Sell!
Sell!
Cash: 2689.05, Shares: 72.98331289562907, Net Value: 9963.0
653
Sell!
Sell!
Sell!
Sell!
Cash: 2788.65, Shares: 71.98331289562907, Net Value: 9959.0
654
Sell!
Sell!
Sell!
Sell!
Cash: 3087.33, Shares: 68.98331289562907, Net Value: 9955.0
655
Buy!
Buy!
Buy!
Buy!
Cash: 2688.77, Shares: 72.98331289562907, Net Value: 9959.0
656
Cash: 2688.77, Shares: 72.98331289562907, Net Value: 9965.0
657
Sell!
Sell!
Sell!
Sell!
Cash: 2888.15, Shares: 70.98331289562907, Net Value: 9964.0
658
Sell!
Sell!
Sell!
Sell!
Cash: 3087.61, Shares: 68.98331289562907, Net Value: 9968.0
659
Sell!
Sell!
Sell!
Sell!
Cash: 3486.37, Shares: 64.98331289562907, Net Value: 9964.0
660
Sell!
Sell!
Sell!
Sell!
Cash: 3586.19, Shares: 63.98331289562907, Net Value: 9973.0
661
Sell!
Sell!
Sell!
Sell!
Cash: 4084.99, Shares: 58.9833128956

...

Buy!
Buy!
Cash: 8971.53, Shares: 14.0, Net Value: 10495.0
9697
Cash: 8971.53, Shares: 14.0, Net Value: 10495.0
9698
Buy!
Buy!
Buy!
Buy!
Cash: 8753.83, Shares: 16.0, Net Value: 10495.0
9699
Sell!
Sell!
Sell!
Sell!
Cash: 10057.87, Shares: 4.0, Net Value: 10493.0
9700
Sell!
Sell!
Sell!
Sell!
Cash: 10383.76, Shares: 1.0, Net Value: 10493.0
9701
Buy!
Buy!
Buy!
Buy!
Cash: 9948.76, Shares: 5.0, Net Value: 10493.0
9702
Cash: 9948.76, Shares: 5.0, Net Value: 10493.0
9703
Sell!
Sell!
Sell!
Sell!
Cash: 10383.68, Shares: 1.0, Net Value: 10493.0
9704
Sell!
Sell!
Sell!
Sell!
Cash: 10492.39, Shares: 0.0, Net Value: 10492.0
9705
Buy!
Buy!
Buy!
Buy!
Cash: 10383.65, Shares: 1.0, Net Value: 10493.0
9706
Buy!
Buy!
Buy!
Buy!
Cash: 9948.53, Shares: 5.0, Net Value: 10493.0
9707
Buy!
Buy!
Buy!
Buy!
Cash: 9621.89, Shares: 8.0, Net Value: 10493.0
9708
Buy!
Buy!
Buy!
Buy!
Cash: 9077.34, Shares: 13.0, Net Value: 10493.0
9709
Cash: 9077.34, Shares: 13.0, Net Value: 10493.0
9710
Sell!
Sell!
Sell!
Sell!
Cash: 9186.2

Buy!
Buy!
Cash: 5685.02, Shares: 44.0, Net Value: 10498.0
9923
Buy!
Buy!
Buy!
Buy!
Cash: 5466.18, Shares: 46.0, Net Value: 10499.0
9924
Buy!
Buy!
Buy!
Buy!
Cash: 5247.26, Shares: 48.0, Net Value: 10501.0
9925
Cash: 5247.26, Shares: 48.0, Net Value: 10503.0
9926
Sell!
Sell!
Sell!
Sell!
Cash: 5356.78, Shares: 47.0, Net Value: 10504.0
9927
Cash: 5356.78, Shares: 47.0, Net Value: 10506.0
9928
Cash: 5356.78, Shares: 47.0, Net Value: 10513.0
9929
Buy!
Buy!
Buy!
Buy!
Cash: 3047.83, Shares: 68.0, Net Value: 10523.0
9930
Buy!
Buy!
Buy!
Buy!
Cash: 2937.86, Shares: 69.0, Net Value: 10523.0
9931
Sell!
Sell!
Sell!
Sell!
Cash: 3377.3, Shares: 65.0, Net Value: 10518.0
9932
Buy!
Buy!
Buy!
Buy!
Cash: 3157.64, Shares: 67.0, Net Value: 10515.0
9933
Buy!
Buy!
Buy!
Buy!
Cash: 2937.82, Shares: 69.0, Net Value: 10519.0
9934
Sell!
Sell!
Sell!
Sell!
Cash: 3047.68, Shares: 68.0, Net Value: 10518.0
9935
Cash: 3047.68, Shares: 68.0, Net Value: 10522.0
9936
Cash: 3047.68, Shares: 68.0, Net Value: 10523.0
9937
Sell

Buy!
Cash: 2820.24, Shares: 70.0, Net Value: 10498.0
10134
Cash: 2820.24, Shares: 70.0, Net Value: 10498.0
10135
Sell!
Sell!
Sell!
Sell!
Cash: 3039.62, Shares: 68.0, Net Value: 10499.0
10136
Sell!
Sell!
Sell!
Sell!
Cash: 3478.18, Shares: 64.0, Net Value: 10495.0
10137
Cash: 3478.18, Shares: 64.0, Net Value: 10496.0
10138
Sell!
Sell!
Sell!
Sell!
Cash: 3697.44, Shares: 62.0, Net Value: 10494.0
10139
Sell!
Sell!
Sell!
Sell!
Cash: 3916.66, Shares: 60.0, Net Value: 10494.0
10140
Cash: 3916.66, Shares: 60.0, Net Value: 10493.0
10141
Buy!
Buy!
Buy!
Buy!
Cash: 3807.04, Shares: 61.0, Net Value: 10493.0
10142
Buy!
Buy!
Buy!
Buy!
Cash: 3587.7, Shares: 63.0, Net Value: 10497.0
10143
Cash: 3587.7, Shares: 63.0, Net Value: 10496.0
10144
Cash: 3587.7, Shares: 63.0, Net Value: 10497.0
10145
Cash: 3587.7, Shares: 63.0, Net Value: 10497.0
10146
Cash: 3587.7, Shares: 63.0, Net Value: 10497.0
10147
Cash: 3587.7, Shares: 63.0, Net Value: 10497.0
10148
Cash: 3587.7, Shares: 63.0, Net Value: 10496.0
10149
Ca

Buy!
Cash: 2543.56, Shares: 72.49496817474525, Net Value: 10591.0
10366
Buy!
Buy!
Buy!
Buy!
Cash: 2432.54, Shares: 73.49496817474525, Net Value: 10592.0
10367
Sell!
Sell!
Sell!
Sell!
Cash: 2543.54, Shares: 72.49496817474525, Net Value: 10591.0
10368
Sell!
Sell!
Sell!
Sell!
Cash: 2765.5, Shares: 70.49496817474525, Net Value: 10590.0
10369
Buy!
Buy!
Buy!
Buy!
Cash: 2543.5, Shares: 72.49496817474525, Net Value: 10590.0
10370
Sell!
Sell!
Sell!
Sell!
Cash: 2654.46, Shares: 71.49496817474525, Net Value: 10587.0
10371
Buy!
Buy!
Buy!
Buy!
Cash: 2543.47, Shares: 72.49496817474525, Net Value: 10588.0
10372
Sell!
Sell!
Sell!
Sell!
Cash: 2654.43, Shares: 71.49496817474525, Net Value: 10587.0
10373
Sell!
Sell!
Sell!
Sell!
Cash: 2876.35, Shares: 69.49496817474525, Net Value: 10587.0
10374
Buy!
Buy!
Buy!
Buy!
Cash: 2543.32, Shares: 72.49496817474525, Net Value: 10590.0
10375
Sell!
Sell!
Sell!
Sell!
Cash: 2876.2, Shares: 69.49496817474525, Net Value: 10587.0
10376
Cash: 2876.2, Shares: 69.494968174745

Buy!
Buy!
Buy!
Buy!
Cash: 1667.81, Shares: 80.3702320908702, Net Value: 10596.0
10613
Cash: 1667.81, Shares: 80.3702320908702, Net Value: 10598.0


In [29]:
cash_open = [d['Open'] for d in cash_list]
shares_open = [d['Open'] for d in shares_list]
shares_value_open = [d['Open'] for d in shares_value_list]
net_value_open = [d['Open'] for d in net_value_list]

In [30]:
def profitChart():
    
    data = [ dict(
            type = 'scatter',
            x = [ii for ii in range(len(pricedata['BidLow']))],
            y = cash_open),
            
            dict(
            type = 'scatter',
            x = [ii for ii in range(len(pricedata['BidLow']))],
            y = shares_value_open),
            
            dict(
            type = 'scatter',
            x = [ii for ii in range(len(pricedata['BidLow']))],
            y = net_value_open)
           ]

    layout = dict(
            title = 'Profit Chart',
            xaxis = dict(
                title = dict(
                    text = 'Time'
                )
            ),
            yaxis = dict(
                title = dict(
                    text = 'Price'
                )
            ))
        
    return go.FigureWidget( data=data, layout=layout)

In [31]:
def profitChart2():
    
    data = [
            dict(
            type = 'scatter',
            x = [ii for ii in range(len(pricedata['BidLow']))],
            y = [nv - net_value_open[0] for nv in net_value_open])
           ]

    layout = dict(
            title = 'Profit Chart',
            xaxis = dict(
                title = dict(
                    text = 'Time'
                )
            ),
            yaxis = dict(
                title = dict(
                    text = 'Price'
                )
            ))
        
    return go.FigureWidget( data=data, layout=layout)

In [32]:
p=profitChart()
p2=profitChart2()

import ipywidgets
ipywidgets.VBox([p,p2])

    'data': [{'type': 'scatter',
              'uid': 'c69dc00d-1622-476c-86ea-8…